Try this one 

https://www.kaggle.com/code/leandrodoze/fuzzy-string-matching-with-hotel-rooms

In [78]:
import spacy
import subprocess
import sys
import jellyfish

try:
    nlp = spacy.load("en_core_web_lg")
except OSError:
    print("Model 'en_core_web_lg' not found. Downloading...")

    # Run the download command
    result = subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_lg"], capture_output=True, text=True)

    # Check if the installation was successful
    if result.returncode == 0:
        print("Download successful. Loading model...")
        nlp = spacy.load("en_core_web_lg")
    else:
        print(f"Download failed: {result.stderr}")
        raise RuntimeError("Could not install 'en_core_web_lg'. Please install manually.")


In [71]:
# Load the data
train_data = pd.read_csv("data/train.csv")

train_data.columns

Index(['Unnamed: 0', 'description_x', 'description_y', 'ticker_x', 'ticker_y',
       'same_security'],
      dtype='object')

In [72]:
def spacy_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)

# Apply similarity function to each row
train_data["spacy"] = train_data.apply(lambda row: spacy_similarity(row["description_x"], row["description_y"]), axis=1)


/var/folders/tx/5kc1s5j56ks8n7b45l51_3vw0000gn/T/ipykernel_22944/4064651327.py:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)


In [ ]:
#Levenshtein distance represents the number of insertions, deletions, 
# and substitutions required to change one word to another.

# Function to compute Levenshtein distance
def compute_levenshtein(text1, text2):
    return jellyfish.levenshtein_distance(text1, text2)

# Apply similarity function to each row
train_data["levenshtein_distance"] = train_data.apply(lambda row: compute_levenshtein(row["description_x"], row["description_y"]), axis=1)

In [ ]:
#A modification of Levenshtein distance, Damerau-Levenshtein 
# distance counts transpositions (such as ifsh for fish) as a single edit.

# Function to compute Damerau-Levenshtein Distance
def compute_dem_levenshtein(text1, text2):
    return jellyfish.damerau_levenshtein_distance(text1, text2)

# Apply similarity function to each row
train_data["dem_levenshtein_distance"] = train_data.apply(lambda row: compute_dem_levenshtein(row["description_x"], row["description_y"]), axis=1)


In [87]:
#Jaro distance is a string-edit distance that gives a floating point response in
#  [0,1] where 0 represents two completely dissimilar strings and 1 represents identical strings.

# Function to compute Levenshtein distance
def comp_jaro_winkler_similarity(text1, text2):
    return jellyfish.jaro_winkler_similarity(text1, text2)

# Apply similarity function to each row
train_data["jaro"] = train_data.apply(lambda row: comp_jaro_winkler_similarity(row["description_x"], row["description_y"]), axis=1)

train_data.head()

,Unnamed: 0,description_x,description_y,ticker_x,ticker_y,same_security,similarity,levenshtein_distance,dem_levenshtein_distance,jaro
0,0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,True,0.769017,11,11,0.896111
1,1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,True,0.896220,30,30,0.835844
2,2,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,False,0.858988,9,9,0.892502
3,3,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,True,0.950788,11,11,0.942637
4,4,visa inc class a,visa inc.,V,V,True,0.822232,8,8,0.877778


In [ ]:
#Create function for fuzzy-wuzzy similarity
def compute_fuzzy_similarity(text1, text2):
    return fuzz.token_sort_ratio(text1, text2)



/var/folders/tx/5kc1s5j56ks8n7b45l51_3vw0000gn/T/ipykernel_22944/2866163739.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)


,Unnamed: 0,description_x,description_y,ticker_x,ticker_y,same_security,similarity,levenshtein_distance,dem_levenshtein_distance,jaro,cosine
0,0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,True,0.769017,11,11,0.896111,0.769017
1,1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,True,0.896220,30,30,0.835844,0.896220
2,2,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,False,0.858988,9,9,0.892502,0.858988
3,3,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,True,0.950788,11,11,0.942637,0.950788
4,4,visa inc class a,visa inc.,V,V,True,0.822232,8,8,0.877778,0.822232
